# CP1 - Vídeo por retroalimentação com Stable Diffusion
**Prazo:** 18/09/2025 23:59  
**Objetivo:** gerar um vídeo onde cada frame é gerado por Stable Diffusion e o frame anterior é usado como entrada (img2img) para o próximo.

---



`PASSO 1: INSTALAÇÃO DE DEPENDÊNCIAS`

In [1]:
!pip install --upgrade diffusers transformers accelerate huggingface_hub

` PASSO 2: IMPORTAÇÕES E CONFIGURAÇÃO`

In [2]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import cv2
import os

In [3]:
# --- Configurações Iniciais ---
# Verifique se o Colab forneceu uma GPU (Vá em "Ambiente de execução" > "Alterar tipo de ambiente de execução" > "Acelerador de hardware" > "GPU")
device = "cuda" if torch.cuda.is_available() else "cpu"
# Nome do modelo no Hugging Face (outras opções: "stabilityai/stable-diffusion-2-1", etc.)
model_id = "runwayml/stable-diffusion-v1-5"
# Diretório para salvar os frames gerados
output_dir = "frames"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
# --- Carregamento dos Modelos ---
# É importante usar torch.float16 para economizar memória VRAM da GPU
print("Carregando modelos... Isso pode levar alguns minutos.")
pipe_txt2img = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe_txt2img = pipe_txt2img.to(device)

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe_img2img = pipe_img2img.to(device)
print("Modelos carregados!")

Carregando modelos... Isso pode levar alguns minutos.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Modelos carregados!




```
PASSO 3: DEFINA SUA ANIMAÇÃO (A PARTE CRIATIVA!)
```



In [5]:
# O prompt descreve a cena. Você pode mudá-lo ao longo do tempo para criar uma história.
prompt = "A beautiful painting of a singular lighthouse, shining its light across a tumultuous sea of churning water, 4k, detailed, trending in artstation"

# --- Parâmetros de Geração ---
num_frames = 100         # Total de quadros do vídeo final. 100 frames a 10 fps = 10 segundos de vídeo.
strength = 0.40          # [EXPERIMENTE!] Força da mudança. Valores baixos (0.2-0.4) para suavidade. Valores altos (0.5-0.7) para mais dinamismo.
guidance_scale = 7.5     # Quão fiel ao prompt o modelo deve ser. Padrão é 7.5.
largura = 512
altura = 512



```
PASSO 4: GERAÇÃO DOS FRAMES
```



In [6]:
print("Iniciando a geração do frame 0 (Text-to-Image)...")
imagem_inicial = pipe_txt2img(prompt, height=altura, width=largura, guidance_scale=guidance_scale).images[0]
imagem_inicial.save(os.path.join(output_dir, "frame_000.png"))
print("Frame 0 salvo!")

Iniciando a geração do frame 0 (Text-to-Image)...


  0%|          | 0/50 [00:00<?, ?it/s]

Frame 0 salvo!


In [7]:
# --- Loop de Geração (Image-to-Image) ---
imagem_anterior = imagem_inicial
caminhos_frames = [os.path.join(output_dir, "frame_000.png")]

for i in range(1, num_frames):
    print(f"Gerando frame {i}/{num_frames-1}...")

    # Gera a nova imagem com base na anterior
    nova_imagem = pipe_img2img(prompt=prompt, image=imagem_anterior, strength=strength, guidance_scale=guidance_scale).images[0]

    # Salva o novo frame
    frame_path = os.path.join(output_dir, f"frame_{i:03d}.png")
    nova_imagem.save(frame_path)
    caminhos_frames.append(frame_path)

    # A imagem gerada agora se torna a imagem de entrada para a próxima iteração
    imagem_anterior = nova_imagem

print("Geração de todos os frames concluída!")

Gerando frame 1/99...


  0%|          | 0/20 [00:00<?, ?it/s]

Gerando frame 2/99...


  0%|          | 0/20 [00:00<?, ?it/s]

Gerando frame 3/99...


...

Gerando frame 98/99...


  0%|          | 0/20 [00:00<?, ?it/s]

Gerando frame 99/99...


  0%|          | 0/20 [00:00<?, ?it/s]

Geração de todos os frames concluída!




```
PASSO 5: MONTAGEM DO VÍDEO FINAL
```



In [8]:
print("Montando o vídeo final...")
video_name = 'video_gerado.mp4'
frame_rate = 10  # Frames por segundo

frame_exemplo = cv2.imread(caminhos_frames[0])
height, width, layers = frame_exemplo.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, frame_rate, (width, height))

for image_path in sorted(caminhos_frames): # Usar sorted() para garantir a ordem
    video.write(cv2.imread(image_path))

video.release()
print(f"Vídeo '{video_name}' salvo com sucesso! Você pode encontrá-lo no painel de arquivos à esquerda.")

Montando o vídeo final...
Vídeo 'video_gerado.mp4' salvo com sucesso! Você pode encontrá-lo no painel de arquivos à esquerda.
